In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import heapq
import pickle
from string import punctuation
from nltk.corpus import stopwords
punctuation = punctuation + '\n'
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
data = pd.read_csv('/content/arabic_categorization_data.csv')

In [ ]:
data.head()

,Unnamed: 0,text,type
0,0,\r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...,culture
1,1,"\r\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحث...",culture
2,2,\r\nاستنكرت إدارة المسرح الوطني التونسي الحملة...,culture
3,3,\r\nاحتضن جناح تونس في القرية الدولية للأفلام ...,culture
4,4,\r\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من ...,culture


In [ ]:
def nltk_summarizer(input_text):
    number_of_sentence = 3
    stopWords = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    word_frequencies = {}
    for word in nltk.word_tokenize(input_text):
        if word not in stopWords:
            if word not in punctuation:
                if word not in word_frequencies.keys():
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

    maximum_frequncy = max(list(word_frequencies.values()),default=3)

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_list = nltk.sent_tokenize(input_text)
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(number_of_sentence, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    return summary

In [ ]:
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text

def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text

def replace_letters(input_text):
    replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    replace = dict((re.escape(k), v) for k, v in replace.items())
    pattern = re.compile("|".join(replace.keys()))
    out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
    return out_text

def clean_text(input_text):
    replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    wnl = nltk.WordNetLemmatizer()
    lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
    out_text = [w for w in lemmatizedTokens if not w in stop_words]
    out_text = ' '.join(out_text)
    return out_text

def stem_text(input_text):
    st = ISRIStemmer()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [st.stem(w) for w in tokens]
    out_text = ' '.join(out_text)
    return out_text


def text_prepare(input_text, ar_text):
    out_text = delete_links(input_text)
    out_text = delete_repeated_characters(out_text)
    out_text = clean_text(out_text)
    out_text = delete_stopwords(out_text)
    if ar_text:
        out_text = replace_letters(out_text)
        out_text = remove_vowelization(out_text)
    else:
        out_text = out_text.lower()
    return out_text

In [ ]:
data['Processed Text'] = data['text'].apply(text_prepare, args=(True,))

In [ ]:
data

,Unnamed: 0,text,type,Processed Text
0,0,\r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1,1,"\r\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحث...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2,2,\r\nاستنكرت إدارة المسرح الوطني التونسي الحملة...,culture,استنكرت اداره المسرح الوطني التونسي الحمله شنه...
3,3,\r\nاحتضن جناح تونس في القرية الدولية للأفلام ...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
4,4,\r\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من ...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
...,...,...,...,...
10361,4814,\r\nأكد المكلف بالإعلام والاتصال بالإدارة العا...,localnews,اكد المكلف بالاعلام والاتصال بالاداره العامه ل...
10362,4815,\r\nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني ...,localnews,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...
10363,4816,\r\nقرر الأهالي بمناطق هيشر وعين القارصي والغو...,localnews,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...
10364,4817,\r\nتمكنت وحدات الحرس الوطني بمحطة الإستخلاص ب...,localnews,تمكنت وحدات الحرس الوطني بمحطه الاستخلاص ببرج ...


In [ ]:
nltk_summarizer
data['summarizer'] = data['text'].apply(nltk_summarizer)

In [ ]:
def aboutsummariz(text:str):
    if len(text)<10:
        ans = 'No'
    else:
        ans = "Yes"
    return ans

In [ ]:
data['done?'] = data['summarizer'].apply(aboutsummariz)

In [ ]:
data

,Unnamed: 0,text,type,Processed Text,summarizer,done?
0,0,\r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...,Yes
1,1,"\r\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحث...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\r\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحث...",Yes
2,2,\r\nاستنكرت إدارة المسرح الوطني التونسي الحملة...,culture,استنكرت اداره المسرح الوطني التونسي الحمله شنه...,,No
3,3,\r\nاحتضن جناح تونس في القرية الدولية للأفلام ...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...,Yes
4,4,\r\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من ...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...,Yes
...,...,...,...,...,...,...
10361,4814,\r\nأكد المكلف بالإعلام والاتصال بالإدارة العا...,localnews,اكد المكلف بالاعلام والاتصال بالاداره العامه ل...,,No
10362,4815,\r\nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني ...,localnews,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...,كما تمّ إلقاء القبض على عنصر رابع (عمره 32 سنة...,Yes
10363,4816,\r\nقرر الأهالي بمناطق هيشر وعين القارصي والغو...,localnews,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...,وتأتي هذه الخطوة على خلفية ما اعتبره أهالي هذه...,Yes
10364,4817,\r\nتمكنت وحدات الحرس الوطني بمحطة الإستخلاص ب...,localnews,تمكنت وحدات الحرس الوطني بمحطه الاستخلاص ببرج ...,\r\nتمكنت وحدات الحرس الوطني بمحطة الإستخلاص ب...,Yes


In [ ]:
import plotly.graph_objects as go

val = dict(data['done?'].value_counts())
labels = list(val.keys())
values = list(val.values())
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

fig.show()

In [ ]:
data = data[data['done?']=='Yes']
final =  data.drop(['Unnamed: 0','done?'],axis=1)

In [ ]:
final.to_csv('summarizdataset.csv',index = False)

In [ ]:
import plotly.graph_objects as go

val = dict(data['done?'].value_counts())
labels = list(val.keys())
values = list(val.values())
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

fig.show()

In [22]:
!pip install transformers datasets sentencepiece sacrebleu


In [33]:
import pandas as pd

# Load the dataset using pandas
df = pd.read_csv('/content/summarizdataset.csv')

# Check the structure of the DataFrame
print(df.head())

# Create a dictionary for training
train_data = {
    'input_text': df['Processed Text'].tolist(),  # Use the preprocessed text
    'target_text': df['summarizer'].tolist()             # Use the summaries
}

# Example: Display a sample
print(train_data['input_text'][:1])
print(train_data['target_text'][:1])


                                                text     type  \
0  \r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...  culture   
1  \r\nتحصل كتاب "المصحف وقراءاته" الذي ألفه باحث...  culture   
2  \r\nاحتضن جناح تونس في القرية الدولية للأفلام ...  culture   
3  \r\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من ...  culture   
4  \r\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوف...  culture   

                                      Processed Text  \
0  اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...   
1  تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...   
2  احتضن جناح تونس القريه الدوليه للافلام بمدينه ...   
3  شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...   
4  نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...   

                                          summarizer  
0  \r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...  
1  \r\nتحصل كتاب "المصحف وقراءاته" الذي ألفه باحث...  
2  تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...  
3  واستأجرت صاحبة المشروع المحامية والكاتبة سيران...

In [24]:
import pyarrow as pa
print(pa.__version__)


14.0.2


In [25]:
!pip install --upgrade pyarrow


In [26]:
!pip install --upgrade datasets
